# Установка библиотек и скачивание датасета

In [ ]:
!pip install simpletransformers

Датасет качать тут - https://russiansuperglue.com/tasks/download/MuSeRC

# Работа с данными

In [ ]:
import json
import re

# Получаем по одному примеру с помощью этого
def cycle_read(doc, ret_label=False):
    with open(doc, 'r', encoding='utf-8') as file:
        # По каждому json
        for line in file:
            json_data = json.loads(line)
            # Чистим текст от номеров предложений
            text = re.sub('([(]\d{1,}[)]\s{0,})', '', json_data['passage']['text'])
            # По каждому вопросу
            for question_data in json_data['passage']['questions']:
                question = question_data["question"]
                # Все ответы
                answers = []
                if ret_label:
                    labels = []
                for answer_data in question_data['answers']:
                    answers.append(answer_data['text'])
                    if ret_label:
                        labels.append(answer_data['label'])
                # Возвращаем данные
                if ret_label:
                    yield text, question, answers, labels
                else:
                    yield text, question, answers

In [ ]:
# import nltk
# nltk.download('punkt')

def get_muserc_sample(text, question, answer):
    # text = nltk.sent_tokenize(text, language='russian')
    ret_str = 'question: ' + question + ' answer: ' + answer + ' paragraph: ' + text

    # for sent in enumerate(text):
    #     ret_str = ret_str + '<b>Sent ' + str(sent[0] + 1) + ': </b>' + sent[1] + '<br> '
    
    return ret_str

In [ ]:
print(get_muserc_sample(
    '(1) Мужская сборная команда Норвегии по биатлону в рамках этапа Кубка мира в немецком Оберхофе выиграла эстафетную гонку. (2) Вторыми стали французы, а бронзу получила немецкая команда. (3) Российские биатлонисты не смогли побороться даже за четвертое место, отстав от норвежцев более чем на две минуты. (4) Это худший результат сборной России в текущем сезоне. (5) Четвёртыми в Оберхофе стали австрийцы. (6) В составе сборной Норвегии на четвёртый этап вышел легендарный Уле-Эйнар Бьорндален. (7) Впрочем, Норвегия с самого начала гонки была в числе лидеров, успешно проведя все четыре этапа. (8) За сборную России в Оберхофе выступали Иван Черезов, Антон Шипулин, Евгений Устюгов и Максим Чудов. (9) Гонка не задалась уже с самого начала: если на стрельбе из положения лежа Черезов был точен, то из положения стоя он допустил несколько промахов, в результате чего ему пришлось бежать один дополнительный круг. (10) После этого отставание российской команды от соперников только увеличивалось. (11) Напомним, что днем ранее российские биатлонистки выиграли свою эстафету. (12) В составе сборной России выступали Анна Богалий-Титовец, Анна Булыгина, Ольга Медведцева и Светлана Слепцова. (13) Они опередили своих основных соперниц - немок - всего на 0,3 секунды.',
    'На сколько секунд женская команда опередила своих соперниц?',
    'Всего на 0,3 секунды.'
))

In [ ]:
print('===== [- ЗАПИСЬ ТРЕНИРОВОЧНЫХ ДАННЫХ В СПИСОК -] =====')

train_samples = []
train_labels = []

for train_sample in cycle_read('../input/muserc/MuSeRC/train.jsonl', ret_label=True):

    for train_answer in train_sample[2]:
        train_samples.append(get_muserc_sample(train_sample[0], train_sample[1], train_answer))
    
    for label in train_sample[3]:
        train_labels.append(int(label))

print('samples-len: ', len(train_samples), '\n',
      'labels-len: ', len(train_labels), sep='')


In [ ]:
train_labels[69], train_samples[69]

In [ ]:
print('===== [- ЗАПИСЬ ПРОВЕРОЧНЫХ ДАННЫХ В СПИСОК -] =====')

val_samples = []
val_labels = []

for val_sample in cycle_read('../input/muserc/MuSeRC/val.jsonl', ret_label=True):

    for val_answer in val_sample[2]:
        val_samples.append(get_muserc_sample(val_sample[0], val_sample[1], val_answer))
    
    for label in val_sample[3]:
        val_labels.append(int(label))

print('samples-len: ', len(val_samples), '\n',
      'labels-len: ', len(val_labels), sep='')


In [ ]:
val_labels[69], val_samples[69]

In [ ]:
import pandas as pd

train_df = pd.DataFrame(zip(train_samples, train_labels) , columns=['text', 'labels'])
val_df = pd.DataFrame(zip(val_samples, val_labels) , columns=['text', 'labels'])

In [ ]:
train_df['labels'].value_counts()

In [ ]:
train_zeroes = train_df[train_df['labels'] == 0][:5382:]
train_ones = train_df[train_df['labels'] == 1][:5382:]

In [ ]:
new_train_df = pd.concat([train_zeroes, train_ones]).sample(frac=1)

In [ ]:
new_train_df['labels'].value_counts()

# Парметры для модели и модель

In [ ]:
import torch
      
device = torch.device("cuda")
print('CUDA DEVICE -', torch.cuda.get_device_name(0))

In [ ]:
from simpletransformers.classification import ClassificationArgs
import logging
model_type = 'xlmroberta'
model_name = "xlm-roberta-base"

model_args = ClassificationArgs()
model_args.num_train_epochs = 8
model_args.learning_rate = 1e-05
model_args.weight_decay = 0.1
model_args.train_batch_size = 8
model_args.max_seq_length = 512

model_args.model_type = model_type
model_args.model_name = model_name

model_args.overwrite_output_dir = True
model_args.fp16 = False
model_args.save_model_every_epoch = False

In [ ]:
model_args

In [ ]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel(model_type, model_name, args=model_args, num_labels=2)

# Тренировка модели

In [ ]:
model.train_model(new_train_df)

# Валидация

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(val_df)

In [ ]:
result

In [ ]:
precision = result['tp'] / (result['tp'] + result['fp'])
recall = result['tp'] / (result['tp'] + result['fn'])
f1 = (2 * precision * recall) / (precision + recall)
print(f"Precisioin: {precision}; Recall: {recall}; F-score: {f1};")

# Создание сабмита для тестовой части

In [ ]:
print('===== [- ЗАПИСЬ ТЕСТОВЫХ ДАННЫХ В СПИСОК -] =====')

test_samples = []

for test_sample in cycle_read('../input/muserc/MuSeRC/test.jsonl', ret_label=False):

    for test_answer in test_sample[2]:
        test_samples.append(get_muserc_sample(test_sample[0], test_sample[1], test_answer))

print('samples-len: ', len(test_samples))


In [ ]:
predictions, raw_outputs = model.predict(test_samples)

In [ ]:
import json

filename = 'MuSeRC_prediction.jsonl'

def save_output(data, doc):
    stroke_data = ''
    i = 0
    with open(doc, 'r', encoding='utf-8') as file:
        # По каждому json
        for line in file:
            json_data = json.loads(line)
            for q in json_data["passage"]["questions"]:
                for a in q["answers"]:
                    a["label"] = data[i]
                    i += 1
            stroke_data += str(json_data) + '\n'
    return stroke_data.replace("'", '"')


with open('./' + filename, 'w') as file:
    file.write(save_output([int(i) for i in predictions], '../input/submission-test/MuSeRC.jsonl'))

In [ ]:
!ls outputs

In [ ]:
from IPython.display import FileLink
from IPython import display

filepaths = [r'./outputs/config.json', r'./outputs/pytorch_model.bin', r'./outputs/tokenizer_config.json', r'./outputs/tokenizer.json', r'./outputs/special_tokens_map.json', r'./outputs/sentencepiece.bpe.model']

for filepath in filepaths:
    display.display(FileLink(filepath))